# 0. Install and Import Dependencies

In [13]:
import cv2
import mediapipe as mp
import numpy as np
import os
import cv2
import numpy as np
import mediapipe as mp
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.utils import to_categorical
mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose

In [2]:
# VIDEO FEED
cap = cv2.VideoCapture(0)
while cap.isOpened():
    ret, frame = cap.read()
    cv2.imshow('Mediapipe Feed', frame)
    
    if cv2.waitKey(10) & 0xFF == ord('q'):
        break
        
cap.release()
cv2.destroyAllWindows()

# 1. Make Detections

In [9]:
def calculate_angle(a,b,c):
    a = np.array(a) # First
    b = np.array(b) # Mid
    c = np.array(c) # End
    
    radians = np.arctan2(c[1]-b[1], c[0]-b[0]) - np.arctan2(a[1]-b[1], a[0]-b[0])
    angle = np.abs(radians*180.0/np.pi)
    
    if angle >180.0:
        angle = 360-angle
        
    return angle 

In [14]:
DATA_PATH = os.path.join('GYM_Data')

# Define the exercises and the number of sequences for each
exercises = ['curl_up', 'curl_down', 'squat_up', 'squat_down']
no_sequences = 30
sequence_length = 30

In [17]:
# Prepare training data (X - features, y - labels)
X, y = [], []
for action in exercises:
    for sequence in range(no_sequences):
        window = []
        for frame_num in range(sequence_length):
            res = np.load(os.path.join(DATA_PATH, action, str(sequence), f"{frame_num}.npy"))
            window.append(res)
        X.append(window)
        y.append(list(exercises).index(action))

X = np.array(X)
y = to_categorical(y).astype(int)

In [20]:
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.05)

model = Sequential()
model.add(LSTM(64, return_sequences=True, activation='relu', input_shape=(30, 33*4)))
model.add(LSTM(128, return_sequences=False, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(len(exercises), activation='softmax'))


model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['categorical_accuracy'])

In [23]:
import cv2
import mediapipe as mp
import numpy as np


# Initialize video capture
cap = cv2.VideoCapture(0)

# Curl counter variables
curl_counter = 0 
curl_stage = None

# Squat counter variables
squat_counter = 0
squat_stage = None

# Initialize MediaPipe pose instance
mp_pose = mp.solutions.pose
mp_drawing = mp.solutions.drawing_utils
with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
    model.load_weights('gym.h5')
    while cap.isOpened():
        ret, frame = cap.read()
        
        # Recolor image to RGB
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False
      
        # Make detection
        results = pose.process(image)
    
        # Recolor back to BGR
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        
        try:
            landmarks = results.pose_landmarks.landmark
            


            # Curl counter logic
            shoulder = [landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x, landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y]
            elbow = [landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].x, landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].y]
            wrist = [landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].x, landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].y]
            curl_angle = calculate_angle(shoulder, elbow, wrist)

            if curl_angle > 160:
                curl_stage = "down"
            if curl_angle < 30 and curl_stage == 'down':
                curl_stage = "up"
                curl_counter += 1

            # Squat counter logic
            hip = [landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].x, landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].y]
            knee = [landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].x, landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].y]
            ankle = [landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].x, landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].y]
            squat_angle = calculate_angle(hip, knee, ankle)

            if squat_angle > 170:
                squat_stage = "up"
            if squat_angle < 90 and squat_stage == "up":
                squat_stage = "down"
                squat_counter += 1
        except:
            pass
        
        # Render curl and squat counters
        cv2.rectangle(image, (0,0), (225,140), (245,117,16), -1)
        
        # Curl counter data
        cv2.putText(image, 'CURL REPS', (15,12), 
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,0,0), 1, cv2.LINE_AA)
        cv2.putText(image, str(curl_counter), 
                    (10,60), 
                    cv2.FONT_HERSHEY_SIMPLEX, 2, (255,255,255), 2, cv2.LINE_AA)
        
        # Squat counter data
        cv2.putText(image, 'SQUAT REPS', (15,82), 
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,0,0), 1, cv2.LINE_AA)
        cv2.putText(image, str(squat_counter), 
                    (10,130), 
                    cv2.FONT_HERSHEY_SIMPLEX, 2, (255,255,255), 2, cv2.LINE_AA)
        
        # Render detections
        # mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
        #                           mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=2), 
        #                           mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2))

        # Display the image
        cv2.imshow('Mediapipe Feed', image)

        # Break gracefully with 'q' key
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

    # Release the capture once all the processing is done.
    cap.release()
    cv2.destroyAllWindows()
    print("The number of curls you did is: ", curl_counter)
    print("The number of squats you did is: ", squat_counter)





The number of curls you did is:  27
The number of squats you did is:  33
